In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import numpy as np 

In [3]:
import os 

notebooks_dir = os.getcwd() 
project_dir = os.path.dirname(notebooks_dir) 
data_raw_dir = os.path.join(project_dir, "data", "raw") 
data_path = os.path.join(data_raw_dir, "OnlineNewsPopularity.csv") 

df = pd.read_csv(data_path)

In [4]:
df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [5]:
df.shape

(39644, 61)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39644 entries, 0 to 39643
Data columns (total 61 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   url                             39644 non-null  object 
 1    timedelta                      39644 non-null  float64
 2    n_tokens_title                 39644 non-null  float64
 3    n_tokens_content               39644 non-null  float64
 4    n_unique_tokens                39644 non-null  float64
 5    n_non_stop_words               39644 non-null  float64
 6    n_non_stop_unique_tokens       39644 non-null  float64
 7    num_hrefs                      39644 non-null  float64
 8    num_self_hrefs                 39644 non-null  float64
 9    num_imgs                       39644 non-null  float64
 10   num_videos                     39644 non-null  float64
 11   average_token_length           39644 non-null  float64
 12   num_keywords                   

In [7]:
df_cleaned = df.drop('url', axis=1)

In [8]:
df_cleaned.columns = [col.strip() for col in df_cleaned.columns] 
print(df_cleaned.columns)

Index(['timedelta', 'n_tokens_title', 'n_tokens_content', 'n_unique_tokens',
       'n_non_stop_words', 'n_non_stop_unique_tokens', 'num_hrefs',
       'num_self_hrefs', 'num_imgs', 'num_videos', 'average_token_length',
       'num_keywords', 'data_channel_is_lifestyle',
       'data_channel_is_entertainment', 'data_channel_is_bus',
       'data_channel_is_socmed', 'data_channel_is_tech',
       'data_channel_is_world', 'kw_min_min', 'kw_max_min', 'kw_avg_min',
       'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg',
       'kw_avg_avg', 'self_reference_min_shares', 'self_reference_max_shares',
       'self_reference_avg_sharess', 'weekday_is_monday', 'weekday_is_tuesday',
       'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday',
       'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend', 'LDA_00',
       'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity',
       'global_sentiment_polarity', 'global_rate_positive_words',
       'glob

In [9]:
from draft.data.sets import pop_target

In [10]:
features, target = pop_target(df_cleaned, 'shares')

In [11]:
from sklearn.preprocessing import StandardScaler 

In [12]:
scaler = StandardScaler() 
scaler.set_output(transform='pandas') 

StandardScaler()

In [13]:
features = scaler.fit_transform(features)

In [14]:
from joblib import dump 

models_dir = os.path.join(project_dir, "models") 
with open(os.path.join(models_dir, "scaler.joblib"), 'wb') as f: 
    dump(scaler, f)

In [15]:
from draft.data.sets import split_sets_random 

In [16]:
X_train, X_val, X_test, y_train, y_val, y_test = split_sets_random(features, target, test_ratio=0.2, as_type='numpy', ret_dict=False)

In [17]:
from draft.data.sets import verify_datasets 

verify_datasets(globals=globals())

    X_train
Type: <class 'numpy.ndarray'>
Dimensions: (23786, 59)

    X_val
Type: <class 'numpy.ndarray'>
Dimensions: (7929, 59)

    X_test
Type: <class 'numpy.ndarray'>
Dimensions: (7929, 59)

    y_train
Type: <class 'numpy.ndarray'>
Dimensions: (23786,)
Value counts: {1: 1, 4: 1, 22: 1, 23: 1, 28: 1, 35: 1, 42: 1, 43: 2, 47: 1, 49: 3, 54: 1, 57: 1, 60: 2, 64: 3, 66: 1, 67: 1, 70: 2, 77: 2, 80: 2, 82: 3, 83: 1, 85: 1, 86: 1, 87: 1, 88: 3, 89: 1, 91: 1, 92: 1, 93: 1, 97: 1, 100: 1, 102: 1, 104: 1, 106: 1, 109: 1, 110: 1, 111: 1, 112: 1, 114: 1, 115: 1, 116: 2, 119: 1, 123: 2, 127: 1, 129: 2, 130: 1, 134: 1, 135: 1, 138: 2, 139: 1, 140: 2, 141: 1, 143: 2, 144: 3, 150: 1, 156: 1, 158: 1, 162: 2, 165: 1, 168: 1, 171: 1, 173: 1, 176: 1, 178: 2, 181: 1, 183: 1, 184: 1, 186: 1, 187: 1, 190: 1, 192: 1, 196: 1, 199: 1, 200: 2, 201: 1, 202: 1, 205: 1, 211: 2, 213: 2, 215: 2, 217: 3, 218: 1, 220: 1, 221: 3, 223: 1, 224: 1, 228: 1, 232: 2, 233: 2, 236: 1, 237: 1, 238: 1, 240: 2, 241: 1, 245: 2